## 🪄 Install `raga-testing-platform` library

In [ ]:
pip install raga-testing-platform==1.1.50

### Put your `ACCESS KEY & SECRET KEY`

In [ ]:
# Define the access key, secret key, and host
ACCESS_KEY = "Paste_your_access_key_here"
SECRET_KEY = "Paste_your_secret_key_here"
HOST = "https://backend.prod3.ragaai.ai"

### Import All raga lib from raga module

In [ ]:
from raga import *
import datetime

### 🪄 Set up a Run

In [ ]:
project_name = "Semantic Segmentation"
run_name = f"Demo-Run-{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"
test_session = TestSession(project_name=project_name,run_name=run_name, access_key=ACCESS_KEY, secret_key=SECRET_KEY, host=HOST)

### 👟 Run Labelling Quality Test

#### Try adding more rules:
```python
rules.add(metric = ""F1Score"",  metric_threshold = 0.25, label = ""ALL"", type=""label"", background_label=""background"", include_background=False)
rules.add(metric = ""Recall"",  metric_threshold = 0.3, label = ""ALL"", type=""label"", background_label=""background"", include_background=True)
rules.add(metric = ""PixelAccuracy"",  metric_threshold = 0.6, label = ""ALL"", type=""label"", background_label=""background"", include_background=True)
rules.add(metric = ""mIoU"",  metric_threshold = 0.2, label = ""ALL"", type=""label"", background_label=""background"", include_background=True)
rules.add(metric = ""wIoU"", metric_threshold = 0.1, weights={""people"": 4, ""vehicles"": 2}, type=""label"", label = ""ALL"")



In [ ]:
rules = LQRules()
rules.add(metric="mistake_score", label=["ALL"], metric_threshold=0.5)


edge_case_detection = labelling_quality_test(test_session=test_session,
                                            dataset_name="cityscape_valv1",
                                            test_name="FMA_SS",
                                            type="labelling_consistency",
                                            output_type="semantic_segmentation",
                                            mistake_score_col_name="MistakeScores",
                                            rules=rules)

test_session.add(edge_case_detection)
test_session.run()

### 👟 Run Data Drift Detection

In [ ]:
rules = DriftDetectionRules()
rules.add(type="anomaly_detection", dist_metric="Mahalanobis", _class="ALL", threshold=26.0)

edge_case_detection = data_drift_detection(test_session=test_session,
                                            test_name="Drift_SS",
                                            train_dataset_name="cityscape_train",
                                            field_dataset_name="cityscape_val",
                                            train_embed_col_name="ImageEmbedding",
                                            field_embed_col_name="ImageEmbedding",
                                            output_type="semantic_segmentation",
                                            level="image",
                                            rules=rules)


test_session.add(edge_case_detection)
test_session.run()

### 👟 Run Failure Mode Analysis - Cluster Level

In [ ]:
rules = FMARules()
rules.add(metric="Precision", metric_threshold=0.8, label="ALL", type="label", background_label="Unlabeled", include_background=False)

cls_default = clustering(test_session=test_session, dataset_name="cityscape_trainv1", method="k-means", embedding_col="ImageEmbedding", level="image", args={"numOfClusters": 5}, interpolation=False, force=True)

edge_case_detection = failure_mode_analysis(test_session=test_session,
                                            dataset_name="cityscape_trainv1",
                                            test_name="LQT_SS",
                                            model="modelA",
                                            gt="GT",
                                            rules=rules,
                                            output_type="semantic_segmentation",
                                            clustering=cls_default,
                                            type="embedding")

test_session.add(edge_case_detection)
test_session.run()

Please switch to https://prod3.ragaai.ai/home/project/runs?projectId=52